In [ ]:
# importing packages
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
from warnings import warn
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import selenium.webdriver.support.ui as ui

#copy paste the below given link and do the filter you want and paste the url again here
driver = webdriver.Chrome('C:/Users/<your-path>/chromedriver.exe')
url = "https://www.linkedin.com/jobs/search?keywords=Data%2BEngineer&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&f_TPR=r2592000&position=1&pageNum=0"
keyword = url.split("keywords=")[1].split("&location")[0].replace("%2B","-")
timestr = time.strftime("%Y%m%d")
driver.get(url)
sleep(3)
action = ActionChains(driver)


#you can change the iteration according to your needs eg: (0,25) or (0,100)
#This code is to scroll and fetch the data
for i in range(0,50):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label = 'Load more results']"))).click()
    except:
        pass

#extract the information 
lists=WebDriverWait(driver, 10).until(lambda x: x.find_elements_by_xpath('//ul[@class="jobs-search__results-list"]/li'))
all_data = []
for li in lists:
    try:
        job_title = WebDriverWait(li, 10).until(lambda x: x.find_element_by_xpath(".//h3[@class='base-search-card__title']")).text
    except:
        job_title = None
    try:
        company_name = WebDriverWait(li, 10).until(lambda x: x.find_element_by_xpath(".//h4[@class='base-search-card__subtitle']")).text
    except:
        company_name = None
    try:
        job_location = WebDriverWait(li, 10).until(lambda x: x.find_element_by_xpath(".//span[@class='job-search-card__location']")).text
    except:
        job_location = None
    try:
        job_id = WebDriverWait(li, 10).until(lambda x: x.find_element_by_xpath("./div[1]")).get_attribute("data-entity-urn")
        job_id = job_id.split("jobPosting:")[1]
    except:
        job_id = None
    try:
        job_url = WebDriverWait(li, 10).until(lambda x: x.find_element_by_xpath(".//a[@class='base-card__full-link']")).get_attribute("href")
    except:
        job_url = None
    try:
        job_posted_dt=WebDriverWait(li, 10).until(lambda x: x.find_element_by_xpath(".//time[@class='job-search-card__listdate']")).get_attribute("datetime")
    except:
        job_posted_dt = None
    result_dict = {"job_title":job_title,
                  "company_name": company_name,
                  "job_location": job_location,
                  "job_id": job_id,
                  "job_url": job_url,
                  "job_posted_dt":job_posted_dt}
    df_1 = pd.DataFrame([result_dict])
    df_1.to_csv('{}_urls_{}.csv'.format(keyword,timestr), mode='a', header=False)
    all_data.append(result_dict)
#         time.sleep(2)

#To Extract further information by navigating to the respective urls
df = pd.DataFrame(all_data)
df = df.drop_duplicates(keep='first')
df.to_csv("{}_intermediate_{}.csv".format(keyword,timestr))
print("URLs extracted from main page...")
seniority_level = []
employment_type = []
job_function = []
industry = []
for url in df["job_url"]:
    try:
        driver.get(url)
        sleep(1)
        action = ActionChains(driver)
    except:
        pass
    try:
        seniority_1 = WebDriverWait(driver, 5).until(lambda x: x.find_element_by_xpath("//*[@id='main-content']/section[1]/section[3]/ul//h3[contains(text(),'Seniority level')]/following-sibling::span")).text
    except:
        seniority_1 = None
    seniority_level.append(seniority_1)
    try:
        employment_1 = WebDriverWait(driver, 5).until(lambda x: x.find_element_by_xpath("//*[@id='main-content']/section[1]/section[3]/ul//h3[contains(text(),'Employment type')]/following-sibling::span")).text
    except:
        employment_1 = None
    employment_type.append(employment_1)
    try:
        job_function_1 = WebDriverWait(driver, 5).until(lambda x: x.find_element_by_xpath("//*[@id='main-content']/section[1]/section[3]/ul//h3[contains(text(),'Job function')]/following-sibling::span")).text
    except:
        job_function_1 = None
    job_function.append(job_function_1)
    try:
        industry_1 = WebDriverWait(driver, 5).until(lambda x: x.find_element_by_xpath("//*[@id='main-content']/section[1]/section[3]/ul//h3[contains(text(),'Industries')]/following-sibling::span")).text
    except:
        industry_1 = None
    industry.append(industry_1)
df["seniority_level"] = seniority_level
df["employment_type"] = employment_type
df["job_function"] = job_function
df["industry"] = industry
df["job_flag"] = keyword
print("Extracted the information completely")
df.to_csv("{}_final_{}.csv".format(keyword,timestr))